<a href="https://colab.research.google.com/github/KozinIlya/KozinIlya/blob/main/beatport_top_100.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Получение данных от beatport

In [1]:
from bs4 import BeautifulSoup  
import pandas as pd  
import numpy as np
import requests
import time
from google.colab import drive
drive.mount('drive', force_remount=True)  

Mounted at drive


In [2]:
url = 'https://www.beatport.com/top-100-releases'
request = requests.get(url)
soup = BeautifulSoup(request.text, 'html.parser') 
request

<Response [200]>

In [3]:
def fun(s):
  t = s.find_all('li',class_="bucket-item ec-item horz-release")
  title_top100 = []
  position_top100 = []
  lable_top100 = []
  link_top100 = []

  for i in t:
    title = i.find('p', class_ = 'buk-horz-release-title').text
    position = i.find('p', class_="buk-horz-release-num").text
    lable = i.find('p', class_="buk-horz-release-labels").text
    link = i.find('a', href=True)['href']
    
    title_top100.append(title)
    position_top100.append(position)
    lable_top100.append(lable)
    link_top100.append(link)
  s = {position:position_top100,title:title_top100,lable:lable_top100,link:link_top100}
  df = pd.DataFrame(s)
  df.columns=['position','title','lable','link']
  df['lable'] = df['lable'].str.replace("\n","")
  df.set_index('position')
  return df

In [4]:
while request.status_code != 200:
  time.sleep(60)
  print('sleep')
df = fun(soup) 

#Сравнение


In [5]:
! gdown --id '1DeNBPMBrO5w4eBgKL7yMfCCkEmXOcdRr'
df_old = pd.read_csv('/content/Beatport_top100_releases.csv')

df_old = df_old.drop(labels=[1,2,3], axis=0)
df_old.columns = (['Unnamed: 0','position_old','title','lable','link'])
df_old.info()

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1DeNBPMBrO5w4eBgKL7yMfCCkEmXOcdRr
To: /content/Beatport_top100_releases.csv
100% 7.14k/7.14k [00:00<00:00, 11.8MB/s]
<class 'pandas.core.frame.DataFrame'>
Int64Index: 97 entries, 0 to 99
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Unnamed: 0    97 non-null     int64 
 1   position_old  97 non-null     int64 
 2   title         97 non-null     object
 3   lable         97 non-null     object
 4   link          97 non-null     object
dtypes: int64(2), object(3)
memory usage: 4.5+ KB


In [6]:
new_releases_df = df.merge(df_old, how='left', on = ['title','lable','link'], indicator=True)\
                  .query("_merge == 'left_only'")\
                  .drop(['_merge','Unnamed: 0','position_old'], axis=1)



In [7]:
n = 0

for index, row in new_releases_df.iterrows():
  t = row['title']
  l = row['lable']
  p = row['position']
  li = 'https://www.beatport.com'+ row['link']
  n = n + 1
  print(f"Название:        {t}\nЛейбл:          {l}\nМесто в Топ100:    {p}\nСсылка: {li}\n\n\n")

if n >0:
  df.to_csv('Beatport_top100_releases.csv')
  !cp Beatport_top100_releases.csv "drive/My Drive/"


Название:        Toolroom Ibiza 2022 Vol. 2
Лейбл:          Toolroom
Место в Топ100:    2
Ссылка: https://www.beatport.com/release/toolroom-ibiza-2022-vol-2/3802318



Название:        Something
Лейбл:          STMPD RCRDS B.V. / Tomorrowland Music
Место в Топ100:    3
Ссылка: https://www.beatport.com/release/something/3824527



Название:        Summer Sampler 2022
Лейбл:          All Day I Dream 
Место в Топ100:    4
Ссылка: https://www.beatport.com/release/summer-sampler-2022/3816126



